# DropPath (Generalized Stochastic Depth) — Teori Notları

Bu notlar DropPath (genelleştirilmiş *Stochastic Depth*) fikrini, amaçlarını ve pratikte nasıl kullanıldığını özetler.  
Odak: **residual/skip bağlantılı ağlarda** *path* seviyesinde düzenlileştirme (regularization).


\
## 1) Dropout → DropPath geçiş motivasyonu

**Dropout** klasik olarak *aktivasyonları* (nöronları) rastgele sıfırlar:  
\[
y = x \odot m,\quad m_i \sim \mathrm{Bernoulli}(p)
\]
- Burada \(p\): *keep probability* (tutma olasılığı).

CNN/ResNet gibi yapılarda kritik bir gözlem var:

- “Tek tek nöron düşürmek” yerine, **bir bloğun (residual branch’in) tamamını** bazen kapatmak daha doğal bir düzenlileştirme sağlar.
- Çünkü ResNet’te öğrenme çoğunlukla “**identity + residual**” kombinasyonu üzerinden yürür.

Bu fikirden **Stochastic Depth** doğar:  
> Eğitim sırasında bazı residual bloklar *rastgele atlanır* (skip yolu açık kalır).

DropPath bunun **genelleştirilmiş** formudur:  
> Sadece “blok atlama” değil, model içindeki farklı *branch/path* bileşenlerini rastgele düşürmeye izin verir.


\
## 2) DropPath nedir?

Tipik residual blok:
\[
y = x + F(x)
\]
DropPath ile eğitim sırasında:
- \(F(x)\) bazen sıfırlanır (*path drop*),
- bazen korunur (*keep*).

Formül:
\[
y = x + \frac{m}{p} F(x),\quad m \sim \mathrm{Bernoulli}(p)
\]

- \(m=0\) ise \(F(x)\) tamamen kapanır → \(y = x\)
- \(m=1\) ise residual korunur ama **\(\frac{1}{p}\)** ile ölçeklenir (beklenen değer korunur).

> \(\frac{m}{p}\) ölçekleme, test zamanında “her şeyi açık” çalıştırırken beklenen aktivasyon seviyesini sabitlemek içindir.

**Test (eval) modunda** DropPath kapalıdır:
\[
y = x + F(x)
\]


\
## 3) Amaç / neyi çözer?

### (a) Daha derin ağlarda genelleme
Derinleştikçe model kapasitesi artar → overfit riski artar.  
DropPath, “aktif derinliği” eğitim boyunca değiştirerek ensemble benzeri etki üretir:

- bazı iterasyonlarda ağ daha sığ
- bazı iterasyonlarda daha derin

Bu, tek bir ağı “çok sayıda alt-ağa” dönüştürür.

### (b) Residual branch’in aşırı güvenilmesini önleme
ResNet’te model bazen residual branch’i fazla agresif kullanır.  
DropPath, residual’a *rastgele kapatma* uygulayarak “identity” yolunun da anlamlı kalmasını sağlar.

### (c) Path/branch tabanlı mimarilerde doğal düzenlileştirme
- ResNeXt (çoklu branch)
- Inception benzeri paralel yollar
- Transformer blokları (MLP + Attention yolları)
DropPath bu yapılarda “branch-level dropout” gibi davranır.


\
## 4) DropPath vs Stochastic Depth

- **Stochastic Depth**: genelde *residual block* seviyesinde “blok düşürme” olarak anlatılır.
- **DropPath**: aynı fikrin daha genel implementasyon ismi (özellikle modern kütüphanelerde).

Pratikte çoğu repo:
- `DropPath` sınıfı yazar
- “stochastic depth” hyperparametresiyle bu sınıfı kullanır.


\
## 5) Drop prob / keep prob nasıl seçilir? (schedule mantığı)

DropPath genelde katman derinliği arttıkça güçlenir:
- erken katmanlar: daha az drop
- geç katmanlar: daha çok drop

Sık kullanılan lineer schedule:
\[
p_\ell = 1 - d \cdot \frac{\ell}{L}
\]
- \(d\): maksimum drop rate (örn. 0.1–0.3)
- \(\ell\): katman indeksi
- \(L\): toplam katman sayısı

Yani “en derindeki bloklar daha sık kapanır”.

> Bu mantık DropBlock’daki “schedule” ile benzer: eğitim/derinlik arttıkça düzenlileştirme artar.


\
## 6) Nereye uygulanır?

### Temel kural
DropPath, **residual toplamından önce** residual branch çıktısına uygulanır.

\[
y = x + \mathrm{DropPath}(F(x))
\]

- `Conv → BN → Act → ... → F(x)` üret
- `F(x)` üstüne DropPath uygula
- `+ x` ile topla

**Not:** Pre-activation ResNet'te de aynı mantık geçerlidir: DropPath residual branch çıkışında, toplama öncesinde durur.


\
## 7) Beklenen davranış

- Train loss genelde biraz daha yüksek çıkar (regularization)
- Test acc daha stabil olabilir
- Çok yüksek drop rate seçilirse underfit görülebilir

Tipik aralıklar:
- CIFAR küçük modeller: \(d \approx 0.05\)–\(0.15\)
- ImageNet büyük modeller: \(d \approx 0.1\)–\(0.3\)


\
## 8) Kısa özet

- DropPath = “residual/path seviyesinde dropout”
- Ana formül: \(x + \frac{m}{p}F(x)\)
- Testte kapalı, eğitimde açık
- Derine gidildikçe drop rate artırmak yaygın


## Minimal Kod: DropPath Modülü (PyTorch)
Aşağıdaki implementasyon, residual branch çıktısına uygulanacak şekilde tasarlanmıştır.

In [ ]:
import torch
import torch.nn as nn

class DropPath(nn.Module):
    """DropPath (Stochastic Depth) - path-level dropout.
    Eğitimde: x * mask / keep_prob
    Testte: x (değişmeden)
    """
    def __init__(self, drop_prob: float = 0.0):
        super().__init__()
        self.drop_prob = float(drop_prob)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.drop_prob == 0.0 or (not self.training):
            return x

        keep_prob = 1.0 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # batch-wise mask
        mask = torch.empty(shape, device=x.device, dtype=x.dtype).bernoulli_(keep_prob)
        return x * mask / keep_prob

## Minimal Kullanım: Residual Blok İçinde
DropPath, residual branch'in çıktısına uygulanır.

In [2]:
import torch.nn.functional as F

class BasicResBlock(nn.Module):
    def __init__(self, cin, cout, stride=1, drop_path=0.0):
        super().__init__()
        self.conv1 = nn.Conv2d(cin, cout, 3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(cout)
        self.act = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(cout, cout, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(cout)

        self.dp = DropPath(drop_path)

        self.skip = nn.Identity()
        if stride != 1 or cin != cout:
            self.skip = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout),
            )

    def forward(self, x):
        identity = self.skip(x)
        out = self.act(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.dp(out)          # <-- DropPath burada
        out = self.act(out + identity)
        return out

# quick check
x = torch.randn(4, 64, 32, 32)
blk = BasicResBlock(64, 64, drop_path=0.1).train()
y = blk(x)
y.shape


torch.Size([4, 64, 32, 32])

## Basit DropPath Schedule (Derinliğe Göre)
Lineer artış örneği: en sonda maksimum drop rate.

In [3]:
def droppath_rate_at(layer_idx: int, num_layers: int, max_drop: float) -> float:
    if num_layers <= 1:
        return 0.0
    return max_drop * (layer_idx / (num_layers - 1))

rates = [droppath_rate_at(i, 6, 0.2) for i in range(6)]
rates


[0.0, 0.04000000000000001, 0.08000000000000002, 0.12, 0.16000000000000003, 0.2]

## Pratik Notlar

- DropPath'i en baştaki katmanlara yüksek vermek genelde stabil değildir; derine doğru artırmak daha iyi çalışır.
- DropPath “spatial bilgi silmez”; bir branch’i tamamen kapatır. Bu yüzden DropBlock/Cutout gibi yöntemlerden farklı davranır.
- Çok yüksek drop rate seçilirse model "aktif derinlik" olarak fazla sığ kalır → underfit.
